<a href="https://colab.research.google.com/github/oddrationale/AdventOfCode2023Python/blob/main/Day11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<article class="day-desc"><h2>--- Day 11: Cosmic Expansion ---</h2><p>You continue following signs for "Hot Springs" and eventually come across an <a href="https://en.wikipedia.org/wiki/Observatory" target="_blank">observatory</a>. The Elf within turns out to be a researcher studying cosmic expansion using the giant telescope here.</p>
<p>He doesn't know anything about the missing machine parts; he's only visiting for this research project. However, he confirms that the hot springs are the next-closest area likely to have people; he'll even take you straight there once he's done with today's observation analysis.</p>
<p>Maybe you can help him with the analysis to speed things up?</p>
<p>The researcher has collected a bunch of data and compiled the data into a single giant <em>image</em> (your puzzle input). The image includes <em>empty space</em> (<code>.</code>) and <em>galaxies</em> (<code>#</code>). For example:</p>
<pre><code>...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#.....
</code></pre>
<p>The researcher is trying to figure out the sum of the lengths of the <em>shortest path between every pair of galaxies</em>. However, there's a catch: the universe expanded in the time it took the light from those galaxies to reach the observatory.</p>
<p>Due to something involving gravitational effects, <em>only some space expands</em>. In fact, the result is that <em>any rows or columns that contain no galaxies</em> should all actually be twice as big.</p>
<p>In the above example, three columns and two rows contain no galaxies:</p>
<pre><code>   v  v  v
 ...#......
 .......#..
 #.........
&gt;..........&lt;
 ......#...
 .#........
 .........#
&gt;..........&lt;
 .......#..
 #...#.....
   ^  ^  ^
</code></pre>
<p>These rows and columns need to be <em>twice as big</em>; the result of cosmic expansion therefore looks like this:</p>
<pre><code>....#........
.........#...
#............
.............
.............
........#....
.#...........
............#
.............
.............
.........#...
#....#.......
</code></pre>
<p>Equipped with this expanded universe, the shortest path between every pair of galaxies can be found. It can help to assign every galaxy a unique number:</p>
<pre><code>....1........
.........2...
3............
.............
.............
........4....
.5...........
............6
.............
.............
.........7...
8....9.......
</code></pre>
<p>In these 9 galaxies, there are <em>36 pairs</em>. Only count each pair once; order within the pair doesn't matter. For each pair, find any shortest path between the two galaxies using only steps that move up, down, left, or right exactly one <code>.</code> or <code>#</code> at a time. (The shortest path between two galaxies is allowed to pass through another galaxy.)</p>
<p>For example, here is one of the shortest paths between galaxies <code>5</code> and <code>9</code>:</p>
<pre><code>....1........
.........2...
3............
.............
.............
........4....
.5...........
.##.........6
..##.........
...##........
....##...7...
8....9.......
</code></pre>
<p>This path has length <code><em>9</em></code> because it takes a minimum of <em>nine steps</em> to get from galaxy <code>5</code> to galaxy <code>9</code> (the eight locations marked <code>#</code> plus the step onto galaxy <code>9</code> itself). Here are some other example shortest path lengths:</p>
<ul>
<li>Between galaxy <code>1</code> and galaxy <code>7</code>: 15</li>
<li>Between galaxy <code>3</code> and galaxy <code>6</code>: 17</li>
<li>Between galaxy <code>8</code> and galaxy <code>9</code>: 5</li>
</ul>
<p>In this example, after expanding the universe, the sum of the shortest path between all 36 pairs of galaxies is <code><em>374</em></code>.</p>
<p>Expand the universe, then find the length of the shortest path between every pair of galaxies. <em>What is the sum of these lengths?</em></p>
</article>

In [1]:
from google.colab import drive
drive.mount(r"/content/drive")

input_file = r"/content/drive/MyDrive/Colab Notebooks/AoC2023/input/11.txt"
with open(input_file, "r") as file:
    input = file.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from bisect import bisect_left
from itertools import combinations

In [3]:
def parse_input(input_str: str) -> list[tuple[int, int]]:
    """
    Parses the input string to find the positions of galaxies.

    Args:
    input_str (str): A multiline string representing the puzzle input.
                     Galaxies are marked with '#', and empty space is marked with '.'.

    Returns:
    list[tuple[int, int]]: A list of tuples, where each tuple contains the (row, column) indices of a galaxy.
    """
    galaxies = []
    rows = input_str.strip().split('\n')

    for i, row in enumerate(rows):
        for j, char in enumerate(row):
            if char == '#':
                galaxies.append((i, j))

    return galaxies

In [4]:
def find_empty_rows_columns(input_str: str) -> tuple[list[int], list[int]]:
    """
    Finds the row and column indexes that do not contain galaxies.

    Args:
    input_str (str): A multiline string representing the puzzle input.

    Returns:
    tuple[list[int], list[int]]: Two lists, the first containing indexes of empty rows
                                  and the second containing indexes of empty columns.
    """
    rows = input_str.strip().split('\n')
    num_rows = len(rows)
    num_cols = len(rows[0])

    # Initialize lists to track galaxy presence in rows and columns
    galaxy_in_row = [False] * num_rows
    galaxy_in_col = [False] * num_cols

    for i, row in enumerate(rows):
        for j, char in enumerate(row):
            if char == '#':
                galaxy_in_row[i] = True
                galaxy_in_col[j] = True

    # Find indexes of empty rows and columns
    empty_rows = [i for i in range(num_rows) if not galaxy_in_row[i]]
    empty_cols = [j for j in range(num_cols) if not galaxy_in_col[j]]

    return empty_rows, empty_cols

In [5]:
def expand_galaxies(galaxies: list[tuple[int, int]],
                    empty_rows: list[int],
                    empty_cols: list[int],
                    expansion_factor: int) -> list[tuple[int, int]]:
    """
    Calculates the new positions of galaxies after expanding the universe using binary search.

    Args:
    galaxies (list[tuple[int, int]]): List of tuples representing the original positions of galaxies.
    empty_rows (list[int]): List of indexes of rows that are empty.
    empty_cols (list[int]): List of indexes of columns that are empty.
    expansion_factor (int): The factor by which empty rows and columns expand.

    Returns:
    list[tuple[int, int]]: A list of tuples representing the new positions of galaxies.
    """
    def get_new_position(old_pos: int, empty_indices: list[int]) -> int:
        """
        Calculates the new position after expansion using binary search.

        Args:
        old_pos (int): The original position (row or column index).
        empty_indices (list[int]): List of empty row or column indexes.

        Returns:
        int: The new position after expansion.
        """
        return old_pos + bisect_left(empty_indices, old_pos) * (expansion_factor - 1)

    new_galaxies = [(get_new_position(row, empty_rows), get_new_position(col, empty_cols))
                    for row, col in galaxies]

    return new_galaxies

In [6]:
def sum_of_manhattan_distances(galaxies: list[tuple[int, int]]) -> int:
    """
    Calculates the sum of Manhattan distances between every unique pair of galaxies.

    Args:
    galaxies (list[tuple[int, int]]): A list of tuples representing the positions of galaxies.

    Returns:
    int: The sum of the Manhattan distances between every unique pair of galaxies.
    """
    def manhattan_distance(pos1: tuple[int, int], pos2: tuple[int, int]) -> int:
        """
        Calculates the Manhattan distance between two points.

        Args:
        pos1, pos2 (tuple[int, int]): The positions of two points.

        Returns:
        int: The Manhattan distance between the points.
        """
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

    distance_sum = sum(manhattan_distance(pair[0], pair[1]) for pair in combinations(galaxies, 2))

    return distance_sum

In [7]:
sum_of_manhattan_distances(
    expand_galaxies(
        parse_input(input),
        *find_empty_rows_columns(input),
        2,
    )
)

9609130

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The galaxies are much <em>older</em> (and thus much <em>farther apart</em>) than the researcher initially estimated.</p>
<p>Now, instead of the expansion you did before, make each empty row or column <em><span title="And you have to have your pinky near your mouth when you do it.">one million</span> times</em> larger. That is, each empty row should be replaced with <code>1000000</code> empty rows, and each empty column should be replaced with <code>1000000</code> empty columns.</p>
<p>(In the example above, if each empty row or column were merely <code>10</code> times larger, the sum of the shortest paths between every pair of galaxies would be <code><em>1030</em></code>. If each empty row or column were merely <code>100</code> times larger, the sum of the shortest paths between every pair of galaxies would be <code><em>8410</em></code>. However, your universe will need to expand far beyond these values.)</p>
<p>Starting with the same initial image, expand the universe according to these new rules, then find the length of the shortest path between every pair of galaxies. <em>What is the sum of these lengths?</em></p>
</article>

In [8]:
sum_of_manhattan_distances(
    expand_galaxies(
        parse_input(input),
        *find_empty_rows_columns(input),
        1000000,
    )
)

702152204842